## Порсонализация предложений для постоянных клиентов интернет магазина "В один клик"

### Описание проекта

Интернет-магазин «В один клик» продаёт разные товары: для детей, для дома, мелкую бытовую технику, косметику и даже продукты. Отчёт магазина за прошлый период показал, что активность покупателей начала снижаться. Привлекать новых клиентов уже не так эффективно: о магазине и так знает большая часть целевой аудитории. Возможный выход — удерживать активность постоянных клиентов. Сделать это можно с помощью персонализированных предложений.

### Задача проекта

Разработать решение, позволяющее персонализировать предложения постоянным клиентам, чтобы увеличить их покупательскую активность.

### Путь решения задачи

* Нужно построить модель, которая предскажет вероятность снижения покупательской активности клиента в следующие три месяца.
* В исследование нужно включить дополнительные данные финансового департамента о прибыльности клиента: какой доход каждый покупатель приносил компании за последние три месяца.
* Используя данные модели и данные о прибыльности клиентов, нужно выделить сегменты покупателей и разработать для них персонализированные предложения.

### Описание данных 

`market_file.csv` - Таблица, которая содержит данные о поведении покупателя на сайте, о коммуникациях с покупателем и его продуктовом поведении.

* id — номер покупателя в корпоративной базе данных.
* Покупательская активность — рассчитанный класс покупательской активности (целевой признак): «снизилась» или «прежний уровень».
* Тип сервиса — уровень сервиса, например «премиум» и «стандарт».
* Разрешить сообщать — информация о том, можно ли присылать покупателю дополнительные предложения о товаре. Согласие на это даёт покупатель.
* Маркет_актив_6_мес — среднемесячное значение маркетинговых коммуникаций компании, которое приходилось на покупателя за последние 6 месяцев. Это значение показывает, какое число рассылок, звонков, показов рекламы и прочего приходилось на клиента.
* Маркет_актив_тек_мес — количество маркетинговых коммуникаций в текущем месяце.
* Длительность — значение, которое показывает, сколько дней прошло с момента регистрации покупателя на сайте.
* Акционные_покупки — среднемесячная доля покупок по акции от общего числа покупок за последние 6 месяцев.
* Популярная_категория — самая популярная категория товаров у покупателя за последние 6 месяцев.
* Средний_просмотр_категорий_за_визит — показывает, сколько в среднем категорий покупатель просмотрел за визит в течение последнего месяца.
* Неоплаченные_продукты_штук_квартал — общее число неоплаченных товаров в корзине за последние 3 месяца.
* Ошибка_сервиса — число сбоев, которые коснулись покупателя во время посещения сайта.
* Страниц_за_визит — среднее количество страниц, которые просмотрел покупатель за один визит на сайт за последние 3 месяца.

`market_money.csv` - Таблица с данными о выручке, которую получает магазин с покупателя, то есть сколько покупатель всего потратил за период взаимодействия с сайтом.

* id — номер покупателя в корпоративной базе данных.
* Период — название периода, во время которого зафиксирована выручка. Например, 'текущий_месяц' или 'предыдущий_месяц'.
* Выручка — сумма выручки за период.

`market_time.csv` - Таблица с данными о времени (в минутах), которое покупатель провёл на сайте в течение периода.

* id — номер покупателя в корпоративной базе данных.
* Период — название периода, во время которого зафиксировано общее время.
* минут — значение времени, проведённого на сайте, в минутах.

`money.csv` - Таблица с данными о среднемесячной прибыли покупателя за последние 3 месяца: какую прибыль получает магазин от продаж каждому покупателю.

* id — номер покупателя в корпоративной базе данных.
* Прибыль — значение прибыли.

## Загрузка данных

In [6]:
!pip install scikit-learn==1.1.3 -q
!pip install shap -q
!pip -q install phik


In [7]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import math
import numpy as np

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, f1_score
import phik
import shap

In [9]:
#Константы
RANDOM_STATE = 42
TEST_SIZE = 0.25

In [10]:
market_file = pd.read_csv('/datasets/market_file.csv')
market_money = pd.read_csv('/datasets/market_money.csv')
market_time = pd.read_csv('/datasets/market_time.csv')
money = pd.read_csv('/datasets/money.csv', sep=';', decimal=",")

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/market_file.csv'

### Общие сведения датасетов

Проверим основную информацию по загруженным данным. Исправлять выявленные недостатки будем в дальнейших шагах проекта.

In [ ]:
market_file.info()

In [ ]:
market_file.head(3)

In [ ]:
market_file.describe()

Столбцы датасета совпадают с описанием. Явных пропусков не видно. Названия некоторых столбцов содержат пробелы, что нужно будет исправить в дальнейшем. Тип данных некоторых столбцов, например `Маркет_актив_6_мес` и `Акционные_покупки` не совпадает с описанием.

In [ ]:
market_money.info()

In [ ]:
market_money.head(3)

In [ ]:
market_money.describe()

Столбцы датасета совпадают с описанием. Явных пропусков не видно. Тип данных столбца `Выручка` не совпадает с описанием. Кроме того есть записи с выручной 0 и с выручкой более 100 000. Возможно, это выбросы, которые в дальнейшем придется удалить. 

In [ ]:
market_time.info()

In [ ]:
market_time.head(3)

In [ ]:
market_time.describe()

Столбцы датасета совпадают с описанием. Явных пропусков не видно. 

In [ ]:
money.info()

In [ ]:
money.head(3)

In [ ]:
money.describe()

Столбцы датасета совпадают с описанием. Пропусков не обнаружено.

### Вывод

Данные были загружены и изучены. Предварительно, в данных есть подозрительные значения: аномалии или выбросы, нужно проверить. Пропуски в данных не обнаружены, обнаружены неверные типы данных.

## Предобработка данных

Теперь займемся предобработкой данных. Приведем в порядок наименования столбцов, приведем типы данных, поищем пропуски, опечатки, и дубли в данных.

### Переименование столбцов

Так как в рамках проекта позволено оставить киррилические наименование столбцов, то просто заменим пробелы на подчеркивания.

In [ ]:
def columns_remove_spaces(df):
    df.columns = df.columns.str.replace(' ', '_').str.lower()

In [ ]:
columns_remove_spaces(market_file)
columns_remove_spaces(market_money)
columns_remove_spaces(market_time)
columns_remove_spaces(money)

In [ ]:
market_file.columns

### Приведение типов данных

Некоторые колонки содержание численны данные были определены как `object` исправим это.

In [ ]:
def object_to_float(df, columns):
    for col in columns:
        df[col] = df[col].astype('float64')

In [ ]:
market_file.head(5)

In [ ]:
market_file.info()

In [ ]:
market_money.info()

### Обработка дублей в данных

Проверим отсутствие явных дубликатов (спойлер: их нет)

In [ ]:
market_file.duplicated().sum()

Но вот если исключить ключевое поле `Id`, то есть 11 дублей. Пока что непонятно, что с ними делать. Пока что оставим как есть. Тем более что записей в датасете всего 1300 - не так уж и много.

В остальных датасетах нет смысла исключать поле Id, проверим только полные дубли.

In [ ]:
print('Число явных дубликатов в датафрейме market_money', market_money.duplicated().sum())
print('Число явных дубликатов в датафрейме market_time', market_time.duplicated().sum())
print('Число явных дубликатов в датафрейме money', money.duplicated().sum())

### Обработка пропусков в данных

Явные пропуски в данных не были обнаружены, надо найти неявные

In [ ]:
def check_string_nan_values(df, columns):
    columns_with_nan = {}
    for col in columns:    
        na_count = df[(df[col].str.lower().isin(['', ' ', 'none', 'nan']))][col].count()
        if na_count > 0:
            columns_with_nan[col] = na_count

    return columns_with_nan

In [ ]:
cols = check_string_nan_values(market_file, market_file.select_dtypes(include=['object']).columns.to_list())
print('Список колонок market_file с пустыми значениями', cols)

In [ ]:
cols = check_string_nan_values(market_money, market_money.select_dtypes(include=['object']).columns.to_list())
print('Список колонок market_money с пустыми значениями', cols)

In [ ]:
cols = check_string_nan_values(market_time, market_time.select_dtypes(include=['object']).columns.to_list())
print('Список колонок market_time с пустыми значениями', cols)

Похоже что датасеты не содержат неявных пропусков


### Вывод

На данном этапе исследования были проделаны следующие шаги.

* Переименованы колонки и приведены в удобный для работы вид
* Выставлены правильные типы данных
* Поиск дубликатов. Они не были обнаружены
* Явные и неявные пропуски не обнаружены

## Исследовательский анализ данных

**market_file**

Теперь займемся исследованием данных. Проверим данные на выбросы и аномальные значения, а так проверим не ускальзнули от нас ли какие-либо ошибки в данных.

In [ ]:

    
def histogram(df, col, target):
    plt.figure(figsize=(8,6))
    plot = sns.histplot(df, bins=20, kde=True, hue=target, x=col)
    plot.set_title(f'Рапределение по {col}', fontsize=16)
    plot.set_ylabel('Количество', fontsize=14)
    
    
def categoral_unique(df, col):
    display(df[col].unique())
    
    plt.figure(figsize=(8,6))
    plot = sns.countplot(y=col, data=df)
    plot.set_title(f'Рапределение по {col}', fontsize=16)
    plot.set_xlabel('Количество', fontsize=14)
    
    
def hist_with_wiskers(df, col, target):
    sns.set()
    f, axes = plt.subplots(1, 2, figsize=(16, 4))
    axes[0].set_title(f'Гистограмма для {col}', fontsize=16)
    axes[0].set_ylabel('Количество', fontsize=14)
    if target != None:
        sns.histplot(df, bins=20, kde=True, ax=axes[0], hue=target, x=col)
    else:
        sns.histplot(df, bins=20, kde=True, ax=axes[0], x=col)
    axes[1].set_title(f'График ящик с усами для {col}', fontsize=16)
    sns.boxplot(data=df, ax=axes[1], y=col)
    axes[1].set_ylabel(col, fontsize=14)
    plt.show()

def pivot_bar_plot(df, col):
    plt.figure(figsize=(8,6))
    plot = sns.barplot(x=col, data=df, y=df.index)
    plot.set_title(f'Рапределение по {col}', fontsize=16)
    plot.set_xlabel('Количество', fontsize=14)

In [ ]:
categoral_unique(market_file, 'покупательская_активность')

Видим некоторый дисбаланс значений в целевом признаке, это нужно будет учесть в модели.

In [ ]:
categoral_unique(market_file, 'тип_сервиса')

Обнаружилась опечатка.  
Так же обратим внимание, что премиум пользователей меньше, что и логично.

In [ ]:
market_file['тип_сервиса'] = market_file['тип_сервиса'].replace('стандартт', 'стандарт')

In [ ]:
categoral_unique(market_file, 'разрешить_сообщать')

In [ ]:
hist_with_wiskers(market_file, 'маркет_актив_6_мес', 'покупательская_активность')

In [ ]:
market_file.query('маркет_актив_6_мес < 2')

In [ ]:
hist_with_wiskers(market_file, 'маркет_актив_тек_мес', 'покупательская_активность')

И снова не выбросы, а удивительные данные

In [ ]:
hist_with_wiskers(market_file, 'длительность', 'покупательская_активность')

Все выглядит хорошо.

In [ ]:
hist_with_wiskers(market_file, 'акционные_покупки', 'покупательская_активность')

Большинство покупателей не ориентируются на скидки. Это звучит логично, люди уже давно просекли фишку ложных скидок и ориентируются на цену в меру своих возможностей. Однако все еще есть те, кто следует скидкам, поэтому при подготовке данных для модели разумным будет разделить пользователей на две части `Часто покупает по акции` и `Редко покупает по акции`, превратив колонку `Акционные_покупки` в категоральный признак.

Также замечен скачок, хотелось сказать, что выбросы, но нет, просто акционные покупки

In [ ]:
categoral_unique(market_file, 'популярная_категория') 

In [ ]:
hist_with_wiskers(market_file, 'средний_просмотр_категорий_за_визит', 'покупательская_активность')

Все выглядит хорошо.

In [ ]:
hist_with_wiskers(market_file, 'неоплаченные_продукты_штук_квартал', 'покупательская_активность')

In [ ]:
hist_with_wiskers(market_file, 'ошибка_сервиса', 'покупательская_активность')

Сайт работает не очень стабильно, это может сказаться на популярности сервиса...

In [ ]:
hist_with_wiskers(market_file, 'страниц_за_визит', 'покупательская_активность')

Все выглядит хорошо.

**market_money**

In [ ]:
categoral_unique(market_money, 'период') 

In [ ]:
market_money_by_period = market_money.pivot_table(index='период', values=['выручка'], aggfunc='sum')
market_money_by_period

In [ ]:
pivot_bar_plot(market_money_by_period, 'выручка') 

Ого прямо ровно ровно.

In [ ]:
hist_with_wiskers(market_money, 'выручка', None)

Замечены выбросы

In [ ]:
market_money.query('выручка > 100000')

Явно выброс, от которого стоит избавиться.

In [ ]:
market_money = market_money[market_money['выручка'] < 100000]

In [ ]:
hist_with_wiskers(market_money, 'выручка', None)

Замечены выбросы

In [ ]:
market_money.query('выручка < 1')

Периодов с нулевой выручкой. Это плохие записи, потому что мы хотим в дальнейшем использовать только тех пользователей, у которых есть активность за три месяца, а такие пустые записи явно нам помешают.

In [ ]:
market_money = market_money[market_money['выручка'] > 0]

In [ ]:
hist_with_wiskers(market_money, 'выручка', None)

"Выбросы" хотела я сказать, но данные распределены равномерно

**market_time**

In [ ]:
market_time_by_period = market_time.pivot_table(index='период', values=['минут'], aggfunc='sum')
market_time_by_period

In [ ]:
pivot_bar_plot(market_time_by_period, 'минут') 

А вот и опечатка.

In [ ]:
market_time['период'] = market_time['период'].replace('предыдцщий_месяц', 'предыдущий_месяц')


In [ ]:
hist_with_wiskers(market_time, 'минут', None)

**money**

In [ ]:
hist_with_wiskers(money, 'прибыль', None)

### Выбор активных пользователей

После того как мы исселодовали данные, нужно отобрать клиентов с активностью не менее трех месяцов. Это значит, что нам нужны записи из `market_money`, где у клиента есть покупки за все три периода `'препредыдущий_месяц', 'текущий_месяц', 'предыдущий_месяц'`.

In [ ]:
agg_dict = {'период': ['count']}
grouped = market_money.groupby('id').agg(agg_dict)
grouped.columns = ['count']

grouped = grouped[grouped['count'] == 3]
market_file = market_file[market_file['id'].isin(grouped.index)]
len(market_file)

Отфильтровалось всего 4 записи.

### Вывод

На данном этапе был проведен исследовательский анализ данных. В ходе работы были исправлены опечатки в данных, которые удалось хорошо найти по составленным графикам. Так же было выявлено небольшое количество явно аномальных значений, которые были отброшены из рассмотрения. В целом данные выглядят хорошо. Так же оставили только пользователей, совершавших покупки в течении трех последних месяцев.

## Объединение таблиц

Теперь объеденим данные из датафреймов `market_file`, `market_money`, `market_time` в одну таблицу.

In [ ]:
market_money_grouped = market_money.pivot_table(index=['id'], columns=["период"])
market_money_grouped.columns = ['выручка_предыдущий_месяц', 'выручка_препредыдущий_месяц', 'выручка_текущий_месяц']
market_money_grouped['id'] = market_money_grouped.index

market_time_grouped = market_time.pivot_table(index=['id'], columns=["период"])
market_time_grouped.columns = ['минут_предыдущий_месяц', 'минут_текущий_месяц']
market_time_grouped['id'] = market_time_grouped.index

market_full = market_file.join(market_money_grouped, on='id', lsuffix="_left", rsuffix="_выручка")
market_full = market_full.rename(columns={'id_left':'id'})
market_full = market_full.join(market_time_grouped, on='id', lsuffix="_left", rsuffix="_минут")
market_full = market_full.rename(columns={'id_left':'id'})
market_full = market_full.drop(['id_выручка', 'id_минут'], axis=1)
market_full.head(5)

### Вывод

Теперь после объеденения датасетов все характеристики в одной таблице.

## Корреляционный анализ

In [ ]:
corr_matrix = market_full.phik_matrix()
plt.figure(figsize=(10, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Матрица корреляции')
plt.show()

Так же построим матрицы корреляции для значений целевого признака отдельно.

In [ ]:
def build_phik_corr_matrix(df, col):
    ax_col = 0
    ax_row = 0
    
    fig, axs = plt.subplots(ncols=1, nrows=df[col].nunique(), figsize=(20,20))
    for i in df[col].unique():
        df_i = df[df[col] == i]
        sns.heatmap(df_i.phik_matrix(), annot=True, cmap='cividis', ax=axs[ax_col])
        axs[ax_col].set_title(i)
        ax_col+=1
        if ax_col > 1:
            ax_col=0
            ax_row+=1  
    fig.tight_layout()
    plt.show()

In [ ]:
build_phik_corr_matrix(market_full, 'покупательская_активность')

### Вывод

Целевым признаком является `Покупательская_активность` и вот список полей, которые имеют хоть корреляцию с ним: `Маркет_актив_6_мес`, `Акционные_покупки`, `Средний_просмотр_категорий_за_визит`, `Неоплаченные_продукты_штук_квартал`, `Страниц_за_визит`, `Выручка_препредыдущий_месяц`, `минут_предыдущий_месяц`, `минут_текущий_месяц`.  
Среди признаков замечана только одна более менее сильная корреляция `Выручка_предыдущий_месяц`, `Выручка_текущий_месяц`. Но значение корреляции меньше 0.9 поэтому не будем удалять признаки.  
Так же график показывает странную корреляцию `Покупательская_активность` и `id`, так что `id` так же уберем из датасета и так как он нам еще понадобиться то превратим его в индекс датасета.

## Использование пайплайнов

Приступим к непосредственному построению модели. Мы переберем модели KNeighborsClassifier(), DecisionTreeClassifier(), LogisticRegression() и  SVC() используя пайплайны.  


Подготовим данные. Закодируем целевой признак в значение 0 и 1. А так же превратим колонку `Акционные_покупки` в категоральный признак.

In [ ]:
market_full['покупательская_активность'] = market_full['покупательская_активность']\
.apply( lambda x: 1 if x=='Снизилась' else 0 )
market_full['покупательская_активность'] = market_full['покупательская_активность'].astype(int)    

In [ ]:
market_full['акционные_покупки_категория'] = market_full['акционные_покупки']\
.apply( lambda x: 'Часто покупает по акции' if x>= 0.5 else 'Редко покупает по акции' )
market_full = market_full.drop(['акционные_покупки'], axis=1)

In [ ]:
market_full = market_full.set_index('id')

In [ ]:
market_full.head()

In [ ]:
# X = market_full.drop(['Покупательская_активность'], axis=1)
X = market_full.drop(['покупательская_активность'], axis=1)
y = market_full['покупательская_активность']
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size = TEST_SIZE, 
    random_state = RANDOM_STATE,
    stratify = y)

In [ ]:
X_train.head()

In [ ]:
ohe_columns = ['разрешить_сообщать', 'популярная_категория', 'тип_сервиса']
ord_columns = ['акционные_покупки_категория']
num_columns = ['маркет_актив_6_мес', 'маркет_актив_тек_мес', 'маркет_актив_тек_мес', 
               'длительность', 'средний_просмотр_категорий_за_визит',
               'неоплаченные_продукты_штук_квартал', 'ошибка_сервиса', 'страниц_за_визит', 
               'выручка_предыдущий_месяц', 'выручка_препредыдущий_месяц', 'выручка_текущий_месяц', 
               'минут_предыдущий_месяц', 'минут_текущий_месяц']

In [ ]:
ohe_pipe = Pipeline(
    [
        ('simpleImputer_ohe', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
        ('ohe', OneHotEncoder(drop = 'first', handle_unknown='error', sparse=False))
    ]
)

In [ ]:
ord_pipe = Pipeline(
    [
        (
            'simple_imputer_ord_before',
            SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        ),
        (
            'ord',
            OrdinalEncoder(categories=[
                                      ['Редко покупает по акции','Часто покупает по акции']],
                          handle_unknown='use_encoded_value',
                          unknown_value=np.nan)
        ),
        (
            'simple_imputer_ord_after',
            SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        )
    ]
)

In [ ]:
data_preprocessor = ColumnTransformer(
    [('ohe', ohe_pipe, ohe_columns),
     ('ord', ord_pipe, ord_columns),
     ('num', MinMaxScaler(), num_columns)
    ], 
    remainder='passthrough'
)

In [ ]:
pipe_final = Pipeline([
    ('preprocessor', data_preprocessor),
    ('models', DecisionTreeClassifier(random_state=RANDOM_STATE))
])

In [ ]:
param_grid = [
    # словарь для модели DecisionTreeClassifier()
    {
        'models': [DecisionTreeClassifier(random_state=RANDOM_STATE)],
        'models__max_depth': range(3, 8),
        'models__max_features': range(2,7),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']  
    },
    
    # словарь для модели KNeighborsClassifier() 
    {
        'models': [KNeighborsClassifier()],
        'models__n_neighbors': range(2,5),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']   
    },

    # словарь для модели LogisticRegression()
    {
        'models': [LogisticRegression(
            random_state=RANDOM_STATE, 
            solver='liblinear', 
            penalty='l1'
        )],
        'models__C': range(1,5),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']  
    },
    # словарь для модели SVC()
    {
        'models': [SVC(probability = True,random_state=RANDOM_STATE, kernel='poly')],
        'models__degree': range(2, 3),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']
    }
]

In [ ]:
randomized_search = RandomizedSearchCV(
    pipe_final, 
    param_grid, 
    cv=5,
    scoring='roc_auc',
    random_state=RANDOM_STATE,
    n_jobs=-1
)

In [ ]:
randomized_search.fit(X_train, y_train)

In [ ]:
print('Лучшая модель и её параметры:\n\n', randomized_search.best_estimator_)
print ('Метрика лучшей модели на тренировочной выборке:', round(randomized_search.best_score_, 2))

In [ ]:
# проверьте работу модели на тестовой выборке
# рассчитайте прогноз на тестовых данных
y_test_pred = randomized_search.predict(X_test)
y_test_proba = randomized_search.predict_proba(X_test)
print(f'Метрика ROC-AUC на тестовой выборке: {round(roc_auc_score(y_test, y_test_proba[:,1]), 2)}')

In [ ]:
print(f'Метрика F1-score на тестовой выборке: {round(f1_score(y_test, y_test_pred, average="macro"), 2)}')

### Вывод

На данном шаге исследования была найдена и обучена модель для определения снижения активности покупателей сервиса. Хорошие результаты показала модель `LogisticRegression(C=2, penalty='l1', random_state=42, solver='liblinear')` - 0.9. Метрика roc_auc для тестовой выборке показла результат 0.92 и является лучшим вариантом. Показатели очень хорошие, наша модель хорошо справляется с предсказанием. Так же метрика f1 показала  0.90.  

Несмотря на дисбаланс в классах, получились высокие значения метрик. Наилучшей метрикой оказалась roc_auc, она более устойчива к изменениям в распределении классов. 

## Анализ важности признаков

In [ ]:
pip install --upgrade shap

In [ ]:

import shap
X_train_2 = pipe_final.named_steps['preprocessor'].fit_transform(X_train)
explainer = shap.LinearExplainer(randomized_search.best_estimator_.named_steps['models'], X_train_2)

X_test_2 = pipe_final.named_steps['preprocessor'].transform(X_test)

feature_names = pipe_final.named_steps['preprocessor'].get_feature_names_out()

X_test_2 = pd.DataFrame(X_test_2, columns=feature_names)
 
shap_values = explainer(X_test_2)
 
#shap.plots.bar(shap_values, max_display=30)

shap.summary_plot(shap_values, plot_size=(20,9), plot_type='bar',)
shap.plots.beeswarm(shap_values, max_display=30)



## Сегментация покупателей


Далее выявим сегмент пользователей и проведем анализ поведения пользователей в рамках этого сегмента. Наша цель - составить рекомендации для заказчика по увеличению активности данного сегмента. 

Были выбраны прибыль и вероятность снижения активности пользователя

In [ ]:
y_test_proba = randomized_search.predict_proba(X_test)[:,1]
y_train_proba = randomized_search.predict_proba(X_train)[:,1]

In [ ]:
X_test.head(3)

In [ ]:
X_test_full = X_test.copy()
X_train_full = X_train.copy()
X_test_full['вероятность_снижения'] = y_test_proba
X_train_full['вероятность_снижения'] = y_train_proba
df_full = pd.concat([X_train_full, X_test_full])

money = money.set_index('id')
df_full = df_full.join(money)


In [ ]:
df_full.head(5)

In [ ]:

fig = plt.figure(figsize=(10,8))
sns.scatterplot(data=df_full, y='прибыль', x='вероятность_снижения')
plt.xlabel('Вероятность снижения активности')
plt.ylabel('Прибыль')
plt.title('Зависимость вероятности снижения активности от выручки')
plt.show()


Судя по диаграмме рассеяния, зависимости между вероятностью снижения активности пользователя и прибыли нет (что по моему мнению очень странно). 

In [ ]:
def build_scatterplots(cat_columns):
    for cat_col in cat_columns:
        fig = plt.figure(figsize=(10,8))
        sns.scatterplot(data=df_full, y='прибыль', x='вероятность_снижения', hue=cat_col)
        plt.xlabel('Вероятность снижения активности')
        plt.ylabel('Прибыль')
        plt.title('Зависимость вероятности снижения активности от выручки')
        plt.show()

In [ ]:
cat_columns = list(df_full.select_dtypes(include='object').columns)
build_scatterplots(cat_columns)

Можно обратить внимание на распределение `Акционные_покупки_категория`. Пользователи, которые часто покупают по скидке чаще всего имеют высокую вероятность снижения активности. Видимо,  сидят ждут черную пятницу.)

Проанализируем этот сегмент. Для этого добавим колонку и в ней промаркируем пользователей, которые часто покупают по акции и имеют вероятность снижения активности более 80%.

In [ ]:
df_full.head(4)

In [ ]:
df_full['сегмент'] = df_full\
.apply( lambda row: 'Исследуемый сегмент' \
        if row['вероятность_снижения'] > 0.8 and row['акционные_покупки_категория']=='Часто покупает по акции' \
        else 'Остальные пользователи' \
      , axis=1)


In [ ]:

categoral_unique(df_full, 'популярная_категория')

Самая популярная категория оказалась "товары для детей" и "домашний текстиль". Обе категории относятся к людям, которые сидят дома и покупают товары домой (мамочки товары для детей, домашний текстиль - уют дома). Также товары для детей крайне дорогие, их выгодно покупать со скидками, как и домашний текстиль.

Чуть менее популярная категория косметики и аксесуаров. В какой-то момент времени косметика на маркетплейсах была действительно дешевле, чем в других магазинах, а также ее ассортимент значительно выше. Возможно, связано с этим

In [ ]:
categoral_unique(df_full, 'тип_сервиса')

Люди по скидке покупают премиум. Наверное, именно поэтому у них есть деньги на премиум, ведь они экономят на всем подряд)

In [ ]:
histogram(df_full, 'страниц_за_визит', 'сегмент')

Видно по пользователи по скидке просматривают меньше страниц. Похоже на то что эти пользователи не ищут и выбирают, а покупают целенаправленно. Или они попадают на скидки, когда ищут что-то конкретное?

In [ ]:
histogram(df_full, 'средний_просмотр_категорий_за_визит', 'сегмент')

А в среднее кол-во просматриваемых категорий не различается.

In [ ]:
histogram(df_full, 'неоплаченные_продукты_штук_квартал', 'сегмент')

In [ ]:
df[df_full['неоплаченные_продукты_штук_квартал'].describe()

Все пользователи склонны скроллить сервис, но не покупать, однако график смещается влево, это говорит о том, что они делают это значительно реже, чем от них ожидается. Возможно, они добавляют в корзины варианты того, что хотят купить, и выбирают лишь один вариант из 2-3 выбранных

Что касается пользователь акционных товаров, есть пик в районе 9-10. Это значит, что эти пользователи чуть более склоны набирать товары в корзину и оставлять их там до лучших времен. 


Теперь сравним количество маркетинговых коммуникаций у покупателей по акции и всего массива пользователей. 

In [ ]:
histogram(df_full, 'маркет_актив_6_мес', 'сегмент')

In [ ]:
histogram(df_full, 'маркет_актив_тек_мес', 'сегмент')

Покупатели по акции получали такое же количество внимания от маркетплейса, как и все остальные пользователи. Это значит что магазин не предоставляет им каких либо дополнительных источников получения информации о скидках. 


Попробуем описать исследуемый нами сегмент. Это пользователи, которые проводят не так много времени на сайте. Они не сравнивают товары, не выбирают лучший, они просмотривают один-два товара и покупают.  

Рекомендации: 
1. Сегментировать рассылки, а значит и сегментировать пользователей. Выявить, какими категориями пользователь интересуется больше и присылать ему информацию по его потребностям
2. Преобразовать главную страницу и разместить на ней скидки на категории, которыми пользователь ранее интересовался. Возможно, не в рамках данного сервиса, а значит подключить SEO и другие источники информации. На основе этой информации можно также улучшить качество рассылок


## Общий вывод



В рамках исследования были проделаны следующие шаги.  
  
- Загрузка данных 

Данные были загружены в датафреймы библиотеки pandas и изучены для дальнейшей работы, намечен план работ

- Предобработка данных

В рамках намеченного плана были, 
1. Были изменены названия столбцов
2. Приведены типы данных в соответствие с типами данных в столбцах
3. Произведен поиск явных и неявных дубликатов, явных и неявных пропусков 

- Исследовательский анализ данных

На этом этапе были построены диаграммы распределения всех признаков. С помощью графиков были обнаружены и исправлены опечатки в категоральных признаках. Также было замечено разделение данных на две группы: `Акционные_покупки` явно выделял две группы пользователей поэтому признак был превращен в категориальный. 

- Корреляционный анализ данных

Целевым признаком является `Покупательская_активность` и вот список полей, которые имеют хоть корреляцию с ним: `Маркет_актив_6_мес`, `Акционные_покупки`, `Средний_просмотр_категорий_за_визит`, `Неоплаченные_продукты_штук_квартал`, `Страниц_за_визит`, `Выручка_препредыдущий_месяц`, `минут_предыдущий_месяц`, `минут_текущий_месяц`.  
Так же график показал сильную зависимость между целевым признаком и `id` так что в дальнейшем при подготовке данных к обучению модели признак идентификатор был удален.
Сильной корреляции между другими признаками не было выявлено поэтому все остальные признаки были оставленны в датасете.

- Объеденение таблиц

Датасеты `market_file`, `market_money`, `market_time` были объеденены в один датафрейм.

- Обучение модели

С использованием пайпланов из библиотеки sklearn были обучены модели KNeighborsClassifier(), DecisionTreeClassifier(), LogisticRegression() и SVC(). При обучении моделей средстави пайплана преебирались некоторое количество гиперпараметров моделей с целью нахождения модели, которая даст лучший результат.  
По результатам обучения лучшей стала модель `LogisticRegression(C=2, penalty='l1', random_state=42, solver='liblinear')`. Метрика roc_auc для тренеровочной выборки 0.90 для тестовой 0.89. Удивительно, но значения метрики имеют такой низкий разброс. И даже наличие дисбаланса в целевом признаке не помешало получить значение метрики f1=0.90.

- Анализ важности признаков и сегментация пользователей

Было описано поведение выделенного сегмента пользователей "пользователи часто покупают по скидке и пользователи имеют высокую вероятность снижения покупательской активности". Это пользователи, которые проводят не так много времени на сайте. Они не сравнивают товары, не выбирают лучший, они просмотривают один-два товара и покупают.

Рекомендации, которые можно дать по улучшению работы с пользователями сегмента "пользователи часто покупают по скидке и пользователи имеют высокую вероятность снижения покупательской активности":

1. Сегментировать рассылки, а значит и сегментировать пользователей. Выявить, какими категориями пользователь интересуется больше и присылать ему информацию по его потребностям
2. Преобразовать главную страницу и разместить на ней скидки на категории, которыми пользователь ранее интересовался. Возможно, не в рамках данного сервиса, а значит подключить SEO и другие источники информации. На основе этой информации можно также улучшить качество рассылок

